In [13]:
# Step 1 import libraries
import numpy as np
import pandas as pd


In [120]:
# Step 2 Import the dataset from the attached file wind.txt
data = pd.read_csv('wind.txt', sep='\s+', na_values='NaN', header=None)
data.columns = data.iloc[0]
data = data[1:]

In [121]:
data

,Yr,Mo,Dy,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
1,61,1,1,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.50,15.04
2,61,1,2,14.71,NaN,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83
3,61,1,3,18.50,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.50,7.67,12.75,12.71
4,61,1,4,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
5,61,1,5,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6570,78,12,27,17.58,16.96,17.62,8.08,13.21,11.67,14.46,15.59,14.04,14.00,17.21,40.08
6571,78,12,28,13.21,5.46,13.46,5.00,8.12,9.42,14.33,16.25,15.25,18.05,21.79,41.46
6572,78,12,29,14.00,10.29,14.42,8.71,9.71,10.54,19.17,12.46,14.50,16.42,18.88,29.58
6573,78,12,30,18.50,14.04,21.29,9.13,12.75,9.71,18.08,12.87,12.46,12.12,14.67,28.79


In [122]:
# Step 3 Assign it to a variable called data and replace the first 3 columns by a proper datetime index.
date_str = data['Yr'].astype(str) + '-' + data['Mo'].astype(str).str.zfill(2) + '-' + data['Dy'].astype(str).str.zfill(2)
data['Date'] = pd.to_datetime(date_str, format='%y-%m-%d')
data = data.drop(['Yr', 'Mo', 'Dy'], axis=1).set_index('Date')
data.head()


,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
Date,,,,,,,,,,,,
2061-01-01,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.50,15.04
2061-01-02,14.71,NaN,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83
2061-01-03,18.50,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.50,7.67,12.75,12.71
2061-01-04,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
2061-01-05,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83


In [123]:
# Step 4 Year 2061? Do we really have data from this year? Create a function to fix it and apply it. 
def fix_year(date):
    if date.year > 2000:
        year = date.year - 100
    else:
        year = date.year
    return pd.to_datetime(year * 10000 + date.month * 100 + date.day, format='%Y%m%d')

data.index = data.index.map(fix_year)
data.head()

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
Date,,,,,,,,,,,,
1961-01-01,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1961-01-02,14.71,NaN,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83
1961-01-03,18.50,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.50,7.67,12.75,12.71
1961-01-04,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
1961-01-05,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83


In [124]:
# Step 5 Set the right dates as the index. Pay attention at the data type, it should be datetime64[ns].
data.index = pd.to_datetime(data.index)

In [125]:
# Step 6 Compute how many values are missing for each location over the entire record.They should be ignored in all calculations below.
missing_values_count = data.isna().sum()
print(missing_values_count)


0
RPT    6
VAL    3
ROS    2
KIL    5
SHA    2
BIR    0
DUB    3
CLA    2
MUL    3
CLO    1
BEL    0
MAL    4
dtype: int64


In [148]:
# Step 7 Compute how many non-missing values there are in total.
non_missing = data.notnull().sum().sum()
print(non_missing)

78857


In [128]:
# Step 8 Calculate the mean windspeeds of the windspeeds over all the locations and all the times. 
# A single number for the entire dataset.
mean_windspeed = data.mean().mean()
print(mean_windspeed)

nan


C:\Users\dell\AppData\Local\Temp\ipykernel_11084\2072026676.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_windspeed = data.mean().mean()


In [130]:
# Step 9 Create a DataFrame called loc_stats and calculate the min, max and mean windspeeds and standard deviations of the windspeeds at each location over all the days A different set of numbers for each location.
loc_stats = pd.DataFrame()
loc_stats['min'] = data.min()
loc_stats['max'] = data.max()
loc_stats['mean'] = data.mean()
loc_stats['std'] = data.std()


C:\Users\dell\AppData\Local\Temp\ipykernel_11084\35499575.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  loc_stats['min'] = data.min()
C:\Users\dell\AppData\Local\Temp\ipykernel_11084\35499575.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  loc_stats['max'] = data.max()
C:\Users\dell\AppData\Local\Temp\ipykernel_11084\35499575.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  loc_stats['mean'] = data.mean()
C:\Users\dell\AppData\Local\Temp\ipykernel_11084\35499575.py:6: Futu

In [131]:
# Step 10 Create a DataFrame called day_stats and calculate the min, max and mean windspeed and standard deviations of the windspeeds across all the locations at each day. A different set of numbers for each day.
day_stats = pd.DataFrame()
day_stats['min'] = data.min(axis=1)
day_stats['max'] = data.max(axis=1)
day_stats['mean'] = data.mean(axis=1)
day_stats['std'] = data.std(axis=1)


C:\Users\dell\AppData\Local\Temp\ipykernel_11084\3366408285.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  day_stats['min'] = data.min(axis=1)
C:\Users\dell\AppData\Local\Temp\ipykernel_11084\3366408285.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  day_stats['max'] = data.max(axis=1)
C:\Users\dell\AppData\Local\Temp\ipykernel_11084\3366408285.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  day_stats['mean'] = data.mean(axis=1)
C:\Users\dell\AppData\Local\Temp\ipykernel_1

In [137]:
# Step 11 Find the average windspeed in January for each location. Treat January 1961 and January 1962 both as January.
january_data = data[data.index.month == 1]
january_means = january_data.mean()

C:\Users\dell\AppData\Local\Temp\ipykernel_11084\2513391469.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  january_means = january_data.mean()


In [140]:
# Step 12 Downsample the record to a yearly frequency for each location.
yearly_data = data.resample('Y').mean()

C:\Users\dell\AppData\Local\Temp\ipykernel_11084\3083576622.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  yearly_data = data.resample('Y').mean()


In [142]:
# Step 13 Downsample the record to a monthly frequency for each location.
monthly_data = data.resample('M').mean()

C:\Users\dell\AppData\Local\Temp\ipykernel_11084\3530724029.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  monthly_data = data.resample('M').mean()


In [143]:
# Step 14 Downsample the record to a weekly frequency for each location.
weekly_data = data.resample('W').mean()


C:\Users\dell\AppData\Local\Temp\ipykernel_11084\2064623854.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  weekly_data = data.resample('W').mean()


In [146]:
# resample data to weekly frequency starting from January 2, 1961
weekly_data = data.resample('W', closed='left', label='left', loffset=pd.DateOffset(days=1)).mean()

# select the first 52 weeks
weekly_data = weekly_data.iloc[:52]

# calculate the desired statistics
weekly_stats = weekly_data.agg(['min', 'max', 'mean', 'std'], axis=1)

C:\Users\dell\AppData\Local\Temp\ipykernel_11084\3748745852.py:2: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  weekly_data = data.resample('W', closed='left', label='left', loffset=pd.DateOffset(days=1)).mean()
C:\Users\dell\AppData\Local\Temp\ipykernel_11084\3748745852.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  weekly_data = data.resample('W', closed='left', label='left', loffset=pd.DateOffset(days=1)).mean()
